In [1]:
import datetime
from time import strftime
from calendar import monthrange
import operator
import sys

import pandas as pd
import pandas_datareader.data as datareader
import numpy as np

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV

import xgboost

import plotly as py
print (py.__version__) # requires version >= 1.9.0\n",
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
sns.set_style('whitegrid')

random_state = 42
np.random.seed(random_state)



3.4.0


In [5]:
def convert_datestr(datestr):
    """Convert a beginning of month date string to end of month
    2018-01-01 -> 2018-01-31"""
    year_str, month_str, day_str = datestr.split('-')
    end_of_month = monthrange(int(year_str), int(month_str))[1]
    return "%s-%s-%02d" % (year_str, month_str, end_of_month)

def get_fred_data(series, start_date, end_date):
    retframe = datareader.DataReader(series, "fred", start_date, end_date)
    # convert dates from start to end of month and set index
    retframe['yyyymmdd'] = retframe.index.strftime('%Y-%m-%d')
    retframe['yyyymmdd'] = [convert_datestr(str) for str in retframe['yyyymmdd']]
    retframe['DATETIME'] = pd.to_datetime(retframe['yyyymmdd'])
    retframe.reset_index(inplace=True)
    return retframe[["DATETIME", series]]

start_date = datetime.datetime(1963, 1, 1)
end_date = datetime.datetime(2017, 12, 31)

TB3MS = get_fred_data("TB3MS", start_date, end_date)
TB3MS.head()

,DATETIME,TB3MS
0,1963-01-31,2.91
1,1963-02-28,2.92
2,1963-03-31,2.89
3,1963-04-30,2.90
4,1963-05-31,2.93


In [6]:
# metrics 

def maxdrawdown(series):
    """max drawdown of a cumulative return series"""
    return (series / series.cummax() -1).min()

# todo: longest period of underperformance, worst vs.index

def portmetrics_monthly(ret, rf):
    "given monthly return, monthly risk-free return as series, return portfolio metrics dataframe"

    annret = (1 + ret).cumprod().iloc[-1] ** (12/len(ret))-1
    print("annualized return: %f" % annret)

    annvol = ret.std() * np.sqrt(12)
    print("annualized volatility: %f" % annvol)

    #annualized excess return
    annexcess = (1 + ret - rf).cumprod().iloc[-1] ** (12/len(ret))-1
    print("annualized excess return: %f" % annexcess)

    # sharpe
    sharpe = annexcess / annvol
    print("Sharpe: %f" % sharpe)

    return pd.DataFrame({"Metric" : ["Annualized return","Annualized volatility","Sharpe"] , 
                         "Value": [annret, annvol, sharpe]})

def sharpe(ret, rf):
    "given monthly return, monthly risk-free return as series, return Sharpe"
    annvol = ret.std() * np.sqrt(12)
    annexcess_series = 1 + ret - rf
    # cumulative excess return at end of series
    annexcess = annexcess_series.cumprod().iloc[-1] ** (12/len(ret))-1    
    sharpe = annexcess / annvol
    return sharpe


def turnover(data):
    """data has gvkeys, datetime, assumed equal weight, datedf = unique dates"""
    datedf = data.groupby("DATETIME").count().reset_index()[["DATETIME","GVKEY"]]
    datedf["EQUALWEIGHT"] = 1/datedf["GVKEY"]
    datedf.head()
    
    turnover_list = []
    for i in range(1, len(datedf["DATETIME"])):
        date0 = datedf["DATETIME"][i-1]
        t0 = data.loc[data["DATETIME"]==date0][["GVKEY", "EQUALWEIGHT"]]

        date1 = datedf["DATETIME"][i]
        t1 = data.loc[data["DATETIME"]==date1][["GVKEY", "EQUALWEIGHT"]]

        turnover_df = t0.merge(t1, on=["GVKEY"], how='outer',suffixes=["_t0","_t1"])    
        turnover_df.fillna(0, inplace=True)
        turnover_list.append(np.abs(turnover_df["EQUALWEIGHT_t0"] - turnover_df["EQUALWEIGHT_t1"]).sum()/2)



In [7]:
data = pd.read_pickle("data.pickle")
data.head()

,DATETIME,GVKEY,IND_CODE,VALUE,MOMENTUM,RET,RET3
1016,1963-01-31,1010,25,1.443624,0.126256,0.047002,0.077724
4326,1963-01-31,1040,6,0.448922,-0.477048,0.170732,0.041148
4620,1963-01-31,1043,41,0.255952,-0.390406,0.049505,-0.056931
5016,1963-01-31,1045,40,1.383475,-0.130926,0.102041,0.082269
8419,1963-01-31,1075,31,0.476919,-0.187948,0.060082,-0.002032


In [8]:
RF=TB3MS.copy()
RF["RF"]= RF["TB3MS"]/12/100
data.merge(RF[["DATETIME","RF"]], on="DATETIME").head()


,DATETIME,GVKEY,IND_CODE,VALUE,MOMENTUM,RET,RET3,RF
0,1963-01-31,1010,25,1.443624,0.126256,0.047002,0.077724,0.002425
1,1963-01-31,1040,6,0.448922,-0.477048,0.170732,0.041148,0.002425
2,1963-01-31,1043,41,0.255952,-0.390406,0.049505,-0.056931,0.002425
3,1963-01-31,1045,40,1.383475,-0.130926,0.102041,0.082269,0.002425
4,1963-01-31,1075,31,0.476919,-0.187948,0.060082,-0.002032,0.002425


In [9]:
uniquedates = sorted(data["DATETIME"].unique())
uniquedates[:10]


[numpy.datetime64('1963-01-31T00:00:00.000000000'),
 numpy.datetime64('1963-02-28T00:00:00.000000000'),
 numpy.datetime64('1963-03-31T00:00:00.000000000'),
 numpy.datetime64('1963-04-30T00:00:00.000000000'),
 numpy.datetime64('1963-05-31T00:00:00.000000000'),
 numpy.datetime64('1963-06-30T00:00:00.000000000'),
 numpy.datetime64('1963-07-31T00:00:00.000000000'),
 numpy.datetime64('1963-08-31T00:00:00.000000000'),
 numpy.datetime64('1963-09-30T00:00:00.000000000'),
 numpy.datetime64('1963-10-31T00:00:00.000000000')]

In [ ]:
num_buckets = 5

def bucket_df_by_col(df, col, num_buckets=num_buckets):
    # compute quantile buckets for df.col
    # assign new column col_RANK containing quantile bucket for each row
    newcol = col + "_RANK"
    # add a random small num to prevent bucket collisions
    df["tempcol"]=df[col] + np.random.uniform(0,0.0000001,len(df))
    # bucket tempcol into num_buckets quantiles
    vals = pd.qcut(df["tempcol"], num_buckets, labels=False)
    df[newcol] = vals
    #df = df.assign(c=col.values)
    return df

col = "MOMENTUM_RANK"

def perf_by_bucket(df, col, rank, perfcol="RET"):
    perf = df.loc[df[col]==rank][["DATETIME", col, perfcol]]\
        .groupby("DATETIME")\
        .mean()\
        .reset_index()
    return perf

def perf_bucket_by_col(data, col, nbuckets=5, rf=RF):

    col_rank = col + "_RANK"

    dflist = [bucket_df_by_col(data[data["DATETIME"]==d], col, nbuckets) for d in uniquedates]
    datanew= pd.concat(dflist)

    reportdict = {'Label': [],
                  'Annualized return' : [],
                  'Annualized volatility' : [],
                  'Sharpe': []}
    retdf = None
    for i in range(nbuckets):
        
        tempdf = perf_by_bucket(datanew, col_rank, i, perfcol="RET")
        # 1-month signal
        tempdf["RET1P"] = 1 + tempdf["RET"]
        tempdf["CUMPERF"] = tempdf["RET1P"].cumprod()
        tempdf["RF"] = rf["RF"]
        #display(tempdf)

        finalcumreturn = list(tempdf["CUMPERF"])[-1]
        annret = (finalcumreturn**(12/len(tempdf))-1)*100
        vol = tempdf["RET"].std() * np.sqrt(12) *100

        reportdict['Label'].append("Quintile %d " % (i))
        reportdict['Annualized return'].append(annret)
        reportdict['Annualized volatility'].append(vol)
        tempdf_sharpe = tempdf.dropna()
        reportdict['Sharpe'].append(sharpe(tempdf_sharpe["RET"], tempdf_sharpe["RF"]))
        
        tempdf.set_index("DATETIME", inplace=True)
        plt.semilogy(tempdf['CUMPERF'], label="Quintile %d (%.1f%% p.a.)" % (i, annret));
        
        if retdf is None:
            retdf = tempdf
        else:
            retdf = pd.concat([retdf, tempdf])
                 
    plt.legend();
    plt.title(col)
    plt.show()

    with pd.option_context('display.float_format', lambda x: "%.2f" % x):
        display(pd.DataFrame(reportdict))

    return retdf
  
def perf_bucket_by_col_3(data, col, nbuckets=5, rf=RF):
    """Same but put 1/3 of port to work for 3 months"""
    
    dates_df=data[["DATETIME"]].groupby(["DATETIME"]).count()
    dates_df["t0"]=dates_df.index
    dates_df["t1"]=dates_df.shift(-1)["t0"]
    dates_df["t2"]=dates_df.shift(-2)["t0"]

    data["DATETIME1"] = dates_df.loc[data["DATETIME"]]["t1"].values
    data["DATETIME2"] = dates_df.loc[data["DATETIME"]]["t2"].values    
    
    col_rank = col + "_RANK"

    dflist = [bucket_df_by_col(data[data["DATETIME"]==d], col, nbuckets) for d in uniquedates]
    datanew= pd.concat(dflist)


    # compute returns by bucket
    reportdict = {'Label': [],
                  'Annualized return' : [],
                  'Annualized volatility' : [],
                  'Sharpe': []}

    for i in range(num_buckets):

        # get returns for this bucket
        T0 = datanew.loc[datanew[col_rank]==i]
        # merge T0 on date +1 to get returns for month 2
        T1 = T0[["DATETIME1", "GVKEY"]].merge(datanew, 
                                              left_on=["DATETIME1", "GVKEY"], 
                                              right_on=["DATETIME", "GVKEY"])[["DATETIME", "GVKEY", "RET"]]
        # month 3
        T2 = T0[["DATETIME2", "GVKEY"]].merge(datanew, 
                                              left_on=["DATETIME2", "GVKEY"], 
                                              right_on=["DATETIME", "GVKEY"])[["DATETIME", "GVKEY", "RET"]]
        # compute means ... every month we put 1/3 portfolio to work for 3 months
        G0 = T0[["DATETIME","RET"]].groupby("DATETIME").mean()
        G1 = T1[["DATETIME","RET"]].groupby("DATETIME").mean()
        G2 = T2[["DATETIME","RET"]].groupby("DATETIME").mean()
        # merge 3 portfolio returns
        G_all = pd.merge(G1,G2,on="DATETIME", how="outer", suffixes=["1", "2"])
        G_all = pd.merge(G_all,G0,on="DATETIME", how="outer") \
            .reset_index() \
            .sort_values("DATETIME") \
            .dropna()
        # average returns by month
        G_all["RET"] = (G_all["RET"] + G_all["RET1"] + G_all["RET2"]) /3
        tempdf = pd.DataFrame(G_all[["DATETIME", "RET"]])

        tempdf["RET1P"] = 1 + tempdf["RET"]
        tempdf["CUMPERF"] = tempdf["RET1P"].cumprod()
        tempdf.reset_index(inplace=True)
        tempdf["RF"] = RF["RF"]

        lastval = list(tempdf["CUMPERF"])[-1]
        annret = (lastval**(12/len(tempdf))-1)*100
        vol = tempdf["CUMPERF"].pct_change().std() * np.sqrt(12) * 100

        reportdict['Label'].append("Quintile %d " % (i))
        reportdict['Annualized return'].append(annret)
        reportdict['Annualized volatility'].append(vol)
        tempdf_sharpe = tempdf.dropna()
        reportdict['Sharpe'].append(sharpe(tempdf_sharpe["RET"], tempdf_sharpe["RF"]))

        tempdf.set_index("DATETIME", inplace=True)
        plt.semilogy(tempdf['CUMPERF'], label="Quintile %d (%.1f%% p.a.)" % (i, annret));

    plt.legend();
    plt.title(col)
    plt.show()

    with pd.option_context('display.float_format', lambda x: "%.2f" % x):
        display(pd.DataFrame(reportdict))

        # chart, calculate sharpe


In [ ]:
col = "VALUE"
value_returns = perf_bucket_by_col(data, col)
value_returns.head()

In [ ]:
data.head()

In [ ]:
# prediction of RET3__RANK is VALUE_RANK
# compute accuracy, confusion_matrix

# compute rank for value and RET3
bucket_df_by_col(data, "VALUE")
bucket_df_by_col(data, "RET")

acc = accuracy_score(data["RET_RANK"], data["VALUE_RANK"])
print ("Accuracy: %0.6f" % acc)

def conf_matrix_heatmap(y_xval, y_xval_pred, title="Xval confusion matrix"):
    conf_matrix = confusion_matrix(y_xval, y_xval_pred)
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(conf_matrix, annot=True, fmt='d')
    plt.ylabel('Actual Quantile')
    plt.xlabel('Predicted Quantile')
    plt.title(title)
    plt.show()

conf_matrix_heatmap(data["RET_RANK"], data["VALUE_RANK"], "VALUE confusion matrix")

In [ ]:
col = "MOMENTUM"
momentum_returns = perf_bucket_by_col(data, col)
momentum_returns.head()

In [ ]:
# prediction of RET3__RANK is MOMENTUM_RANK
# compute accuracy, confusion_matrix

# make sure you have good rank for momentum and RET3
bucket_df_by_col(data, "MOMENTUM")
bucket_df_by_col(data, "RET")

acc = accuracy_score(data["RET_RANK"], data["MOMENTUM_RANK"])
print ("Accuracy: %0.6f" % acc)

conf_matrix_heatmap(data["RET_RANK"], data["MOMENTUM_RANK"], "MOMENTUM confusion matrix")

In [ ]:
# merge value_returns and momentum_returns by date, bucket
# compute metrics for combined portfolio

combined_returns = value_returns.reset_index().merge(momentum_returns, 
                                                     left_on=["DATETIME","VALUE_RANK"], 
                                                     right_on=["DATETIME", "MOMENTUM_RANK"])

reportdict = {'Label': [],
              'Annualized return' : [],
              'Annualized volatility' : [],
              'Sharpe': []}

for i in range(num_buckets):
        
        tempdf = combined_returns.loc[combined_returns["VALUE_RANK"] == i]
        tempdf["RET1P"]=(tempdf["RET1P_x"] + tempdf["RET1P_y"])/2
        tempdf["RET1"] = tempdf["RET1P"]-1
        tempdf["CUMPERF"] = tempdf["RET1P"].cumprod()
        tempdf.reset_index(inplace=True)
        tempdf["RF"] = RF["RF"]

        lastval = list(tempdf["CUMPERF"])[-1]

        annret = (lastval**(12/len(tempdf))-1)*100
        vol = tempdf["CUMPERF"].pct_change().std() * np.sqrt(12) * 100

        reportdict['Label'].append("Quintile %d " % (i))
        reportdict['Annualized return'].append(annret)
        reportdict['Annualized volatility'].append(vol)
        tempdf_sharpe = tempdf.dropna()
        reportdict['Sharpe'].append(sharpe(tempdf_sharpe["RET1"], tempdf_sharpe["RF"]))

        tempdf.set_index("DATETIME", inplace=True)
        plt.semilogy(tempdf['CUMPERF'], label="Quintile %d (%.1f%% p.a.)" % (i, annret));

        
plt.legend();
plt.title("COMBINED")
plt.show()

with pd.option_context('display.float_format', lambda x: "%.2f" % x):
    display(pd.DataFrame(reportdict))


In [ ]:
# dummy variable for financials, enable model to treat them differently, e.g. p/b
data['FINANCIAL'] = 0
data.loc[data["IND_CODE"]==44,"FINANCIAL"] = 1
data.loc[data["IND_CODE"]==45,"FINANCIAL"] = 1
data.loc[data["IND_CODE"]==46,"FINANCIAL"] = 1
data.loc[data["IND_CODE"]==47,"FINANCIAL"] = 1

In [ ]:
# make 3-month return quintile buckets
bucket_df_by_col(data, "RET3", num_buckets=num_buckets)
   
data.head()

In [ ]:
FIRST_TRAIN_MONTHS=120

# BacktestModel training harness
# initialize with data, sklearn model, some params
# fit_predict: fit from start to some month, predict n months thereafter
# gen_predictions: run fit_predict from some beginning month to end of dataframe using some stride

class BacktestModel():
    
    def __init__(self, 
                 df,
                 create_model=None,
                 startindex=FIRST_TRAIN_MONTHS,
                 scaler=None,
                 fit_missing=None):

        # 1st column = dates
        # 2nd column = gvkeys
        self.data = df.copy()
        self.data.sort_values(["DATETIME", "GVKEY"], inplace=True)
        self.data.reset_index(inplace=True)
        self.data["index"] = self.data.index
        self.uniquedates = self.data[["DATETIME","index"]].groupby("DATETIME").first().reset_index()
        self.uniquedatelist = list(self.uniquedates["index"])
        self.uniquedatedict = dict(zip(self.uniquedates["DATETIME"], 
                                       self.uniquedates["index"]))
        # last column = target
        self.y = df.iloc[:,-1]
        
        # middle = features
        self.X = df.iloc[:,2:-1]
        self.Xrows, self.Xcols = self.X.shape
        
        # create predictions
        self.P = np.zeros(self.y.shape)
        
        self.Xscale = self.X.copy()
        self.yscale = self.y.copy()

        if scaler:
            self.Xscale = scaler().fit_transform(self.Xscale)
            #            self.yscale = scaler().fit_transform(self.yscale.values.reshape(self.Xrows,1))
        
        self.create_model = create_model
        self.startindex = startindex
        self.fit_missing = fit_missing
        
    def fit_predict(self, train_months, predict_months=1, verbose=False):
        """for backtest, train model  
        train on first ntrain rows. if ntrain=121, fit 0:120
        predict following npredict rows 
        if npredict=1, predict row 121
        if npredict=12, predict rows 121-132
        """
        
        # fit first ntrain months
        train_stop_index = self.uniquedatelist[train_months]
        if verbose:
            print("Training months 0:%d, indexes 0:%d" % (train_months-1, train_stop_index-1))
        X_fit = self.Xscale[:train_stop_index]  # e.g. 0:120
        y_fit = self.yscale[:train_stop_index]

        # train model
        self.model = self.create_model()
        self.model.fit(X_fit, y_fit)

        # predict npredict months (but don't exceed bounds)
        if train_months+predict_months > len(self.uniquedatelist)-1:
            predict_stop_index = len(self.Xscale)
        else:
            predict_stop_index = self.uniquedatelist[train_months+predict_months]
            
        if verbose:
            print("Predicting months %d:%d, indexes %d:%d" % (train_months, train_months+predict_months-1, 
                                                              train_stop_index, predict_stop_index-1))
        self.X_predict = self.Xscale[train_stop_index:predict_stop_index] # 121-122
        self.y_predict_proba = self.model.predict_proba(self.X_predict)
        self.y_predict = self.y_predict_proba.dot(np.matrix("0 ; 1; 2; 3; 4"))
        self.P[train_stop_index:predict_stop_index] = self.y_predict.reshape(self.y_predict.shape[0])
        return self.P
    
    def gen_predictions(self,
                        step=1, 
                        splits=None,
                        verbose=False):
        """predict all months, pass either step (# months) or total # of splits"""
        print("%s Starting training" % (strftime("%H:%M:%S")))

        if splits:
            month_indexes = splits[:-1] # last index is nrows
        else:
            # create list of steps
            month_indexes = list(range(self.startindex, len(self.uniquedatelist), step))
            
        steps = [month_indexes[i+1]-month_indexes[i] for i in range(len(month_indexes)-1)]
        # last step -> end
        steps.append(len(self.uniquedatelist) - month_indexes[-1])
        
        if verbose:
            print ("Months: " + str(month_indexes))
            print ("Steps: " + str(steps))

        progress_i = 0
            
        for month_index, forecast_rows in zip(month_indexes, steps):
            if verbose:
                print("Training on first %d months (%d:%d), storing predictions in rows %s" % (month_index, 
                                                                                            0, month_index-1, 
                                                                                            str(range(month_index,month_index+forecast_rows))))
            predictions = self.fit_predict(month_index, forecast_rows, verbose=verbose)
            sys.stdout.write('.')
            progress_i += 1
            if progress_i % 80 == 0:
                print("")
                print("%s Still training step %d of %d" % (strftime("%H:%M:%S"), progress_i, len(month_indexes)))
            sys.stdout.flush()
        print("")    
        print("%s Finished" % (strftime("%H:%M:%S")))


In [ ]:
def my_create_model():
    return LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=10000,
                              n_jobs=-1)

backtestmodel = BacktestModel(data[["DATETIME","GVKEY","FINANCIAL","VALUE","MOMENTUM","RET3_RANK"]],
                              create_model=my_create_model,
                              startindex=FIRST_TRAIN_MONTHS)
backtestmodel.gen_predictions(verbose=False)

In [ ]:
col = "WALKFORWARD_LOGISTIC"
data[col] = backtestmodel.P
perf_bucket_by_col_3(data, col)
#tbh expected better performance than either predictor alone


In [ ]:
TUNEFOLD_LENGTH = 6

# tuned sequentially
#n_estimators: number of base learner trees
n_estimatorss=[50,100,150,200,300,500,1000]
n_estimators = 100

#max_depth: max depth per base tree
#max_depths = range(3,16)
max_depths = range(3,21)
max_depth = 14

#subsample: row subsampling rate (similar to RF)
subsamples = np.linspace(0.75, 0.9, 4)
subsample = 0.7

#colsample_bytree: feature subsampling rate (similar to RF)
#colsample_bytrees = np.linspace(0.4, 0.9, 11)
# only 3 features though
colsample_bytrees = [1/3, 2/3, 1.0]
colsample_bytree = 1/3

#learning_rate: shrinkage factor applied to each base tree update
#learning_rates = np.logspace(-2, 0, 5)
learning_rate = 0.1

#gammas = [0, 1, 5]
gamma = 1

for colsample_bytree in colsample_bytrees:
    
    print("n_estimators: %d" % n_estimators)
    print("max_depth: %d" % max_depth)
    print("subsample: %f" % subsample)
    print("colsample_bytree: %f" % colsample_bytree)
    print("gamma: %f" % gamma)
    print("learning_rate: %f" % learning_rate)
    
    def my_create_model():
        return xgboost.sklearn.XGBClassifier(n_estimators=n_estimators, 
                                             max_depth=max_depth, 
                                             subsample=subsample,
                                             min_child_weight=1,
                                             colsample_bytree=colsample_bytree,
                                             learning_rate=learning_rate,
                                             gamma=gamma,
                                             n_jobs=-1
                                            )

    backtestmodel = BacktestModel(data[["DATETIME","GVKEY","VALUE","MOMENTUM","FINANCIAL","RET3_RANK"]],
                                  create_model=my_create_model,
                                  startindex=FIRST_TRAIN_MONTHS)
    backtestmodel.gen_predictions(verbose=False, step=TUNEFOLD_LENGTH)

    col = "WALKFORWARD_XGB_CLASSIFIER"
    data[col] = backtestmodel.P
    z = perf_bucket_by_col(data, col)


In [ ]:
n_estimators = 300
max_depth = 14
subsample = 0.7
colsample_bytree = 1/3
learning_rate = 0.1
gamma = 0
    
print("n_estimators: %d" % n_estimators)
print("max_depth: %d" % max_depth)
print("subsample: %f" % subsample)
print("colsample_bytree: %f" % colsample_bytree)
print("gamma: %f" % gamma)
print("learning_rate: %f" % learning_rate)

def my_create_model():
    return xgboost.sklearn.XGBClassifier(n_estimators=n_estimators, 
                                         max_depth=max_depth, 
                                         subsample=subsample,
                                         min_child_weight=1,
                                         colsample_bytree=colsample_bytree,
                                         learning_rate=learning_rate,
                                         gamma=gamma,
                                         n_jobs=-1
                                        )

backtestmodel = BacktestModel(data[["DATETIME","GVKEY","VALUE","MOMENTUM","FINANCIAL","RET3_RANK"]],
                              create_model=my_create_model,
                              startindex=FIRST_TRAIN_MONTHS)
backtestmodel.gen_predictions(verbose=False, step=1)

col = "WALKFORWARD_XGB_CLASSIFIER"
data[col] = backtestmodel.P
perf_bucket_by_col_3(data, col)


In [ ]:
dateindexes[120]


In [ ]:
data_confmat = data.copy()
data_confmat["WALKFORWARD_XGB"] = backtestmodel.P
data_confmat = data_confmat[dateindexes[120]:].copy()

# compute rank for pred and RET3
bucket_df_by_col(data_confmat, "WALKFORWARD_XGB")
bucket_df_by_col(data_confmat, "RET3")

acc = accuracy_score(data_confmat["RET3_RANK"], data_confmat["WALKFORWARD_XGB_RANK"])
print ("Accuracy: %0.6f" % acc)

conf_matrix_heatmap(data_confmat["RET3_RANK"], data_confmat["WALKFORWARD_XGB_RANK"], "Gradient Boosting Classifier Confusion Matrix")

In [12]:
# rewrite backtest in more brain-dead way to make sure no error 
n_estimators = 300
max_depth = 14
subsample = 0.7
colsample_bytree = 1/3
learning_rate = 0.1
gamma = 0

# make sure sorted by date, gvkey
data2 = data.sort_values(["DATETIME", "GVKEY"]) \
    .reset_index()\
    .reset_index()[["level_0","DATETIME","GVKEY","FINANCIAL","VALUE","MOMENTUM","RET","RET3_RANK","RET3"]]
data2.rename(index=str, columns={"level_0": "index"}, inplace=True)

# map dates to first row
dateindexes = data2.groupby(["DATETIME"])["index"].first()

P = np.zeros(len(data2))

KeyError: "['FINANCIAL' 'RET3_RANK'] not in index"

In [11]:
for i in range(120, len(dateindexes)):
    
    daterange = dateindexes.index[0:i]
    print("fit from %s to %s" % (str(daterange[0]), str(daterange[-1])))
    fitrange = data2.iloc[0:dateindexes[i],:]
    print("fit from index %d to index %d" % (fitrange.iloc[0,:]["index"], fitrange.iloc[-1,:]["index"]))
    X = fitrange[["VALUE","MOMENTUM","FINANCIAL"]]
    y = fitrange["RET3_RANK"]
    
#     model = LogisticRegression(multi_class='multinomial', 
#                              solver='lbfgs', 
#                              max_iter=10000,
#                              n_jobs=-1)
    model = xgboost.sklearn.XGBClassifier(n_estimators=n_estimators, 
                                        max_depth=max_depth, 
                                        subsample=subsample,
                                        min_child_weight=1,
                                        colsample_bytree=colsample_bytree,
                                        learning_rate=learning_rate,
                                        gamma=gamma,
                                        n_jobs=-1
                                       )
    model.fit(X, y)
    
    print("predict %s" % (dateindexes.index[i]))
    if i+1 >= len(dateindexes): # last start date, predict from there to the end
        predrange = data2.iloc[dateindexes[i]:,:]
    else:
        predrange = data2.iloc[dateindexes[i]:dateindexes[i+1],:]
    print("predict from index %d to index %d" % (predrange.iloc[0,:]["index"], predrange.iloc[-1,:]["index"]))
    X_predict = predrange[["VALUE","MOMENTUM","FINANCIAL"]]
    y_predict_proba = model.predict_proba(X_predict)
    # weighted average 
    y_predict = y_predict_proba.dot(np.matrix("0 ; 1; 2; 3; 4"))
    if i+1 >= len(dateindexes):
        P[dateindexes[i]:] = y_predict.reshape(y_predict.shape[0])
    else:
        P[dateindexes[i]:dateindexes[i+1]] = y_predict.reshape(y_predict.shape[0])

NameError: name 'dateindexes' is not defined

In [ ]:
dateindexes

In [ ]:
# we now have forecasts for full period
# bucket forecasts

# put P forecast into data2
data2["P"] = P  

# bucket predictions, put into PREDRANK
data3 = None
for d in dateindexes.index[120:]:
    tempdf = data2.loc[data2["DATETIME"]==d]
    print(d, len(tempdf))
    tempdf["PREDRANK"] = pd.qcut(tempdf["P"], num_buckets, labels=False)
    if data3 is None:
        data3 = tempdf
    else:
        data3 = pd.concat([data3, tempdf])

In [ ]:
# recall that we predicted 3-month returns
# for all the 1s, we will collect this month's return + next 2 months, to hold for 3 months [datetime, gvkey, return, monthno]
# for now, store the date values as extra columns in data3

dates_df = pd.DataFrame(dateindexes)
dates_df["t0"]=dates_df.index
dates_df["t1"]=dates_df.shift(-1)["t0"]
dates_df["t2"]=dates_df.shift(-2)["t0"]

data3["DATETIME1"] = dates_df.loc[data3["DATETIME"]]["t1"].values
data3["DATETIME2"] = dates_df.loc[data3["DATETIME"]]["t2"].values

In [ ]:
# compute returns by bucket
# calculate sharpe

reportdict = {'Label': [],
              'Annualized return' : [],
              'Annualized volatility' : [],
              'Sharpe': []}

for i in range(num_buckets):

    # get returns for this bucket
    T0 = data3.loc[data3["PREDRANK"]==i]
    # merge T0 on date +1 to get returns for month 2
    T1 = T0[["DATETIME1", "GVKEY"]].merge(data3, 
                                          left_on=["DATETIME1", "GVKEY"], 
                                          right_on=["DATETIME", "GVKEY"])[["DATETIME", "GVKEY", "RET"]]
    # month 3
    T2 = T0[["DATETIME2", "GVKEY"]].merge(data3, 
                                          left_on=["DATETIME2", "GVKEY"], 
                                          right_on=["DATETIME", "GVKEY"])[["DATETIME", "GVKEY", "RET"]]
    # compute means ... every month we put 1/3 portfolio to work for 3 months
    G0 = T0[["DATETIME","RET"]].groupby("DATETIME").mean()
    G1 = T1[["DATETIME","RET"]].groupby("DATETIME").mean()
    G2 = T2[["DATETIME","RET"]].groupby("DATETIME").mean()
    # merge 3 portfolio returns
    G_all = pd.merge(G1,G2,on="DATETIME", how="outer", suffixes=["1", "2"])
    G_all = pd.merge(G_all,G0,on="DATETIME", how="outer") \
        .reset_index() \
        .sort_values("DATETIME") \
        .dropna()
    # average returns by month
    G_all["RET"] = (G_all["RET"] + G_all["RET1"] + G_all["RET2"]) /3
    tempdf = pd.DataFrame(G_all[["DATETIME", "RET"]])

    tempdf["RET1P"] = 1 + tempdf["RET"]
    tempdf["CUMPERF"] = tempdf["RET1P"].cumprod()
    tempdf.reset_index(inplace=True)
    tempdf["RF"] = RF["RF"]

    lastval = list(tempdf["CUMPERF"])[-1]
    annret = (lastval**(12/len(tempdf))-1)*100
    vol = tempdf["CUMPERF"].pct_change().std() * np.sqrt(12) * 100

    reportdict['Label'].append("Quintile %d " % (i))
    reportdict['Annualized return'].append(annret)
    reportdict['Annualized volatility'].append(vol)
    tempdf_sharpe = tempdf.dropna()
    reportdict['Sharpe'].append(sharpe(tempdf_sharpe["RET"], tempdf_sharpe["RF"]))

    tempdf.set_index("DATETIME", inplace=True)
    plt.semilogy(tempdf['CUMPERF'], label="Quintile %d (%.1f%% p.a.)" % (i, annret));

plt.legend();
plt.title("XGBCLASSIFIER")
plt.show()

with pd.option_context('display.float_format', lambda x: "%.2f" % x):
    display(pd.DataFrame(reportdict))
    

In [ ]:
    G0 = T0[["DATETIME","RET"]].groupby("DATETIME").mean()
    G1 = T1[["DATETIME","RET"]].groupby("DATETIME").mean()
    G2 = T2[["DATETIME","RET"]].groupby("DATETIME").mean()
    G_all = pd.merge(G1,G2,on="DATETIME", how="outer", suffixes=["1", "2"])
    G_all = pd.merge(G_all,G0,on="DATETIME", how="outer") \
        .reset_index() \
        .sort_values("DATETIME") \
        .dropna()
    G_all["RET_COMB"] = (G_all["RET"] + G_all["RET1"] + G_all["RET2"]) /3
    G_all["RET"] = G_all["RET_COMB"]
    tempdf = pd.DataFrame(G_all[["DATETIME", "RET"]])
    tempdf


In [ ]:

col = "XGB_CLASSIFIER"
data[col] = backtestmodel.P
z = perf_bucket_by_col(data, col)
display(HTML(z.to_html()))


In [ ]:
data.describe()

In [ ]:
# prediction of RET3_RANK is VALUE_RANK
# compute accuracy, confusion_matrix

print (len(data)/25)
# make sure p is in data
bucket_df_by_col(data, col)
bucket_df_by_col(data, "RET3")

acc = accuracy_score(data["RET3_RANK"], data[col+"_RANK"])
print ("Accuracy: %0.6f" % acc)

conf_matrix_heatmap(data["RET3_RANK"], data[col + "_RANK"])

In [ ]:
# plot feature importances
import operator

featimpdict = backtestmodel.model._Booster.get_score(importance_type='gain')

xgb_featlabels=[]
xgb_featimps = []

for key, val in reversed(sorted(featimpdict.items(), key=operator.itemgetter(1))):
    xgb_featlabels.append(key)
    xgb_featimps.append(val)

def plotly_feat_imp(labels, featimps, name):

    trace1 = Bar(
        x=labels[:30],
        y=featimps[:30],
        name=name,
    )

    zlayout = Layout(
        title='%s Feature Importance' % (name),        
        xaxis = dict(title = "", 
                     tickangle=30,
                     tickfont=dict(
                         size=10,
                         color='black'
                     )),
        yaxis = dict(title = "Importance", 
                    ),
        
        margin=layout.Margin(
            b=150,
        ),    
        barmode='group',
    )

    data = [trace1]
    
    fig = Figure(data=data, layout=zlayout)
    iplot(fig)

plotly_feat_imp(xgb_featlabels, xgb_featimps, "XGBoost")

In [ ]:
# rewrite backtest in more brain-dead way to make sure no error 
n_estimators = 300
max_depth = 14
subsample = 0.7
colsample_bytree = 1/3
learning_rate = 0.1
gamma = 0

# make sure sorted by date, gvkey
data2 = data.sort_values(["DATETIME", "GVKEY"]) \
    .reset_index()\
    .reset_index()[["level_0","DATETIME","GVKEY","FINANCIAL","VALUE","MOMENTUM","RET","RET3_RANK","RET3"]]
data2.rename(index=str, columns={"level_0": "index"}, inplace=True)

# map dates to first row
dateindexes = data2.groupby(["DATETIME"])["index"].first()

P = np.zeros(len(data2))

for i in range(120, len(dateindexes)):
    
    daterange = dateindexes.index[0:i]
    print("fit from %s to %s" % (str(daterange[0]), str(daterange[-1])))
    fitrange = data2.iloc[0:dateindexes[i],:]
    print("fit from index %d to index %d" % (fitrange.iloc[0,:]["index"], fitrange.iloc[-1,:]["index"]))
    X = fitrange[["VALUE","MOMENTUM","FINANCIAL"]]
    y = fitrange["RET3_RANK"]
    
#     model = LogisticRegression(multi_class='multinomial', 
#                              solver='lbfgs', 
#                              max_iter=10000,
#                              n_jobs=-1)
    model = xgboost.sklearn.XGBClassifier(n_estimators=n_estimators, 
                                        max_depth=max_depth, 
                                        subsample=subsample,
                                        min_child_weight=1,
                                        colsample_bytree=colsample_bytree,
                                        learning_rate=learning_rate,
                                        gamma=gamma,
                                        n_jobs=-1
                                       )
    model.fit(X, y)
    
    print("predict %s" % (dateindexes.index[i]))
    if i+1 >= len(dateindexes): # last start date, predict from there to the end
        predrange = data2.iloc[dateindexes[i]:,:]
    else:
        predrange = data2.iloc[dateindexes[i]:dateindexes[i+1],:]
    print("predict from index %d to index %d" % (predrange.iloc[0,:]["index"], predrange.iloc[-1,:]["index"]))
    X_predict = predrange[["VALUE","MOMENTUM","FINANCIAL"]]
    y_predict_proba = model.predict_proba(X_predict)
    # weighted average 
    y_predict = y_predict_proba.dot(np.matrix("0 ; 1; 2; 3; 4"))
    if i+1 >= len(dateindexes):
        P[dateindexes[i]:] = y_predict.reshape(y_predict.shape[0])
    else:
        P[dateindexes[i]:dateindexes[i+1]] = y_predict.reshape(y_predict.shape[0])

In [ ]:
list(range(backtestmodel.startindex, len(backtestmodel.uniquedatelist), 108))

In [ ]:
120
len(uniquedates)

In [ ]:
(658-120)/5

In [ ]:
list(range(16,21))

In [ ]:
        mydata = data.copy()
        mydata.sort_values(["DATETIME", "GVKEY"], inplace=True)
        mydata.reset_index(inplace=True)
        mydata["index"] = mydata.index
        myuniquedates = mydata.groupby("DATETIME").index.first().reset_index()
        myuniquedatelist = list(myuniquedates["index"])
        myuniquedatedict = dict(zip(myuniquedates["DATETIME"], 
                                       myuniquedates["index"]))


In [ ]:
myuniquedates = mydata.groupby("DATETIME").index.first().reset_index()
myuniquedates